In [1]:
# Basic Libraries
import pandas as pd
import numpy as np
import random
import seaborn as sb
import matplotlib.pyplot as plt
import googlemaps

from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import UnexpectedAlertPresentException

import pickle

In [2]:
# Loading Data
users = pd.read_csv('userDf_new_V2.csv')
data = pd.read_csv('Final_list.csv')
data_new = pd.read_csv('Final_list.csv')

In [3]:
# Generating empty columns for time to travel from towns to the restaurants
for i in users.address.unique():
    data[f'Travelling from {i}'] = None

In [4]:
users.address.unique()

array(['Jurong East', 'Bedok', 'Woodlands', 'Geylang', 'Jurong West',
       'Bishan', 'Bukit Timah', 'Toa Payoh', 'Punggol', 'Tampines',
       'Bukit Panjang', 'Sembawang', 'Kallang/Whampoa', 'Bukit Merah',
       'Choa Chu Kang', 'Marine Parade', 'Bukit Batok', 'Hougang',
       'Ang Mo Kio', 'Sengkang', 'Clementi', 'Serangoon', 'Yishun',
       'Pasir Ris', 'Queenstown', 'Central Area'], dtype=object)

In [5]:
data.head()

,placeId,Stall Name,Cuisine,Cuisine 1,Cuisine 2,Cuisine 3,Shop Type,Address,Google Address,Average Review,...,Travelling from Bukit Batok,Travelling from Hougang,Travelling from Ang Mo Kio,Travelling from Sengkang,Travelling from Clementi,Travelling from Serangoon,Travelling from Yishun,Travelling from Pasir Ris,Travelling from Queenstown,Travelling from Central Area
0,0,Fatty Cheong,[None],NaN,NaN,NaN,Kiosk or Stall,#01-144 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-120, Singapore 150006",4.2,...,None,None,None,None,None,None,None,None,None,None
1,1,Mani Iyer Mess,['South Indian'],South Indian,NaN,NaN,Restaurant,#01-01 Little India Conservation Area Hindoo R...,Hindoo Rd,4.3,...,None,None,None,None,None,None,None,None,None,None
2,2,Stall #01-46,"['Chinese', 'Snacks']",Chinese,Snacks,NaN,Kiosk or Stall,#01-46 Jalan Batu Food Centre Jalan Batu 432004,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,3,Wei Xiang Kitchen,"['Chinese', 'Seafood']",Chinese,Seafood,NaN,Kiosk or Stall,#01-106 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-106, Singapore 150006",5.0,...,None,None,None,None,None,None,None,None,None,None
4,4,The Thai Table,"['Asian', 'Indochinese', 'Thai']",Asian,Indochinese,Thai,Restaurant,Picardy Garden Jalan Pari Burong 488686,"20 Jln Pari Burong, Singapore 488686",4.2,...,None,None,None,None,None,None,None,None,None,None


In [6]:
# Creating the travel data class

class travel_data():
    def __init__(self, driving_time, driving_distance, public_transport):
        self.drive_time = driving_time
        self.drive_distance = driving_distance
        self.public_time = public_transport
        
    def driving_distance(self):
        return self.drive_distance
    
    def driving_time(self):
        return self.drive_time
    
    def public_transport_time(self):
        return self.public_time

### First attempt using Google Maps API 

In [7]:
# Function to use on apply to see if a postal code is present
def argument(x):
    try:
        digit = (x.split()[-1])
        if digit.isdigit():
            return digit
        else:
            return None
    except:
        return None

In [8]:
def get_distance(start_point, end_point, travel_type):

    gmaps = googlemaps.Client(key = "GoogleAPIKey")
    
    if travel_type == 1:
        mode_ = "driving"
    else:
        mode_ = "transit"
    
    matrix = gmaps.distance_matrix(origins = start_point, destinations = end_point, mode = mode_)
    #print(matrix)
    distance = float(matrix['rows'][0]['elements'][0]['distance']['text'].split()[0])
    
    time = matrix['rows'][0]['elements'][0]['duration']['text']

    time = time.split()
    if(len(time) == 4):
        time = int(time[0])*60 + int(time[2])
    else:
        time = int(time[0])
        
    return time, distance

In [9]:
# Creating the postal code column
data['postal'] = data.Address.apply(lambda x: argument(x))

In [10]:
# Getting all unique postal codes present
groups = list(data.groupby("postal").groups.keys())
group_length = len(groups)

### API KEY EXPIRED: SECOND ATTEMPT, ADDING ALL THOSE THAT WERE NOT ADDED

In [11]:
missing_distance = data[data["Travelling from Central Area"].isna()]

In [12]:
missing_postals = missing_distance.postal.unique()

In [13]:
missing_distance.loc[missing_distance.postal == missing_postals[0]]

,placeId,Stall Name,Cuisine,Cuisine 1,Cuisine 2,Cuisine 3,Shop Type,Address,Google Address,Average Review,...,Travelling from Hougang,Travelling from Ang Mo Kio,Travelling from Sengkang,Travelling from Clementi,Travelling from Serangoon,Travelling from Yishun,Travelling from Pasir Ris,Travelling from Queenstown,Travelling from Central Area,postal
0,0,Fatty Cheong,[None],NaN,NaN,NaN,Kiosk or Stall,#01-144 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-120, Singapore 150006",4.2,...,None,None,None,None,None,None,None,None,None,150006
3,3,Wei Xiang Kitchen,"['Chinese', 'Seafood']",Chinese,Seafood,NaN,Kiosk or Stall,#01-106 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-106, Singapore 150006",5.0,...,None,None,None,None,None,None,None,None,None,150006
10,10,M.J Cafe Corner (Special Ginger Tea),"['Drinks', 'Juices']",Drinks,Juices,NaN,Kiosk or Stall,#01-09 ABC Brickworks Food Centre Jalan Bukit ...,"6 Jalan Bukit Merah, #01-09, Singapore 150006",NaN,...,None,None,None,None,None,None,None,None,None,150006
11,11,Soon Huat Satay Mee Hoon,"['Asian', 'Chinese']",Asian,Chinese,NaN,Kiosk or Stall,#01-47 Abc Brickworks Market And Food Centre J...,"6 Jalan Bukit Merah, #01-47 ABC Food Centre, S...",4.3,...,None,None,None,None,None,None,None,None,None,150006
16,16,Amoy Street Famous 16 Prawn Noodle,['Chinese'],Chinese,NaN,NaN,Kiosk or Stall,#01-126 ABC Brickworks Food Centre Jalan Bukit...,"7 Maxwell Rd, Singapore 069111",NaN,...,None,None,None,None,None,None,None,None,None,150006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,387,Pin Wei Local Kitchen,"['Asian', 'Chinese', 'Local']",Asian,Chinese,Local,Kiosk or Stall,#01-122 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, Singapore 150006",5.0,...,None,None,None,None,None,None,None,None,None,150006
396,396,Mohd Zaid,"['Chinese', 'Desserts', 'Malaysian']",Chinese,Desserts,Malaysian,Kiosk or Stall,#01-11 ABC Brickworks Food Centre Jalan Bukit ...,"1 Jalan Bukit Merah, Singapore 150001",4.6,...,None,None,None,None,None,None,None,None,None,150006
399,399,Tanglin,"['Asian', 'Chinese', 'Local']",Asian,Chinese,Local,Kiosk or Stall,#01-130 ABC Brickworks Food Centre Jalan Bukit...,NaN,NaN,...,None,None,None,None,None,None,None,None,None,150006
405,405,Western Barbeque,['Western'],Western,NaN,NaN,Kiosk or Stall,#01-18 ABC Brickworks Food Centre Jalan Bukit ...,"51 Old Airport Rd, #01-53, Singapore 390051",4.0,...,None,None,None,None,None,None,None,None,None,150006


In [14]:
len(missing_postals)
count = 0

### THIRD ATTEMPT: GETTING DISTANCE WITH SELENIUM FOR THOSE THE API COULD NOT FIND PROPERLY
- VERY SLOW BUT MORE ACCURATE

In [15]:
def get_distance2(driver, start_point, end_point):
    
    url_link = f"https://www.google.com.sg/maps/dir/{start_point}/{end_point}"
    driver.get(url_link)
    sleep(1)
    travel_modes = WebDriverWait(driver,2).until(
        EC.presence_of_all_elements_located((By.XPATH, ("//button"))))
    
    # 2: Car, 3: Public Transport, 4: Walking
    
    travel_modes[2].click()
    sleep(2)
    car_features = WebDriverWait(driver,5).until(
        EC.presence_of_element_located((By.XPATH, ("//div[@class='section-directions-trip-numbers']")))).text
    
    travel_modes[3].click()
    sleep(2)
    bus_features = WebDriverWait(driver,5).until(
        EC.presence_of_element_located((By.XPATH, ("//div[@class='section-directions-trip-numbers']")))).text
    
    car_features = car_features.split("\n")
    
    car_time = car_features[0]
    car_distance= car_features[1]
    
    car_time = car_time.split()
    
    if len(car_time) == 4:
        car_time = car_time[0] * 60 + car_time[2]
    else:
        car_time = car_time[0]
    car_distance = car_distance.split()[0]
    
    bus_features = bus_features.split()
    if len(bus_features) == 4:
        bus_features = bus_features[0] * 60 + bus_features[2]
    else:
        bus_features = bus_features[0]
    
    return travel_data(car_time, car_distance, bus_features)

In [16]:
data

,placeId,Stall Name,Cuisine,Cuisine 1,Cuisine 2,Cuisine 3,Shop Type,Address,Google Address,Average Review,...,Travelling from Hougang,Travelling from Ang Mo Kio,Travelling from Sengkang,Travelling from Clementi,Travelling from Serangoon,Travelling from Yishun,Travelling from Pasir Ris,Travelling from Queenstown,Travelling from Central Area,postal
0,0,Fatty Cheong,[None],NaN,NaN,NaN,Kiosk or Stall,#01-144 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-120, Singapore 150006",4.2,...,None,None,None,None,None,None,None,None,None,150006
1,1,Mani Iyer Mess,['South Indian'],South Indian,NaN,NaN,Restaurant,#01-01 Little India Conservation Area Hindoo R...,Hindoo Rd,4.3,...,None,None,None,None,None,None,None,None,None,209107
2,2,Stall #01-46,"['Chinese', 'Snacks']",Chinese,Snacks,NaN,Kiosk or Stall,#01-46 Jalan Batu Food Centre Jalan Batu 432004,NaN,NaN,...,None,None,None,None,None,None,None,None,None,432004
3,3,Wei Xiang Kitchen,"['Chinese', 'Seafood']",Chinese,Seafood,NaN,Kiosk or Stall,#01-106 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-106, Singapore 150006",5.0,...,None,None,None,None,None,None,None,None,None,150006
4,4,The Thai Table,"['Asian', 'Indochinese', 'Thai']",Asian,Indochinese,Thai,Restaurant,Picardy Garden Jalan Pari Burong 488686,"20 Jln Pari Burong, Singapore 488686",4.2,...,None,None,None,None,None,None,None,None,None,488686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20844,20844,Snackz It! (Vivo City),"['Asian', 'Local', 'Snacks']",Asian,Local,Snacks,Confectionery or Bakery,#B2-K24 Vivocity Harbourfront Walk 098585,"1 HarbourFront Walk, #B2-K24 VivoCity, Singapo...",2.0,...,None,None,None,None,None,None,None,None,None,098585
20845,20845,LiHO (Wisma Atria),"['Drinks', 'Juices', 'Singaporean']",Drinks,Juices,Singaporean,Kiosk or Stall,#B1-03 Wisma Atria Orchard Road 238877,"#B1-03, Wisma Atria, 435 Orchard Road, 238877",3.7,...,None,None,None,None,None,None,None,None,None,238877
20846,20846,Taste of Thailand,"['Asian', 'Indochinese', 'Thai']",Asian,Indochinese,Thai,Restaurant,#01-1001 Yishun Industrial Food Centre Yishun ...,"1001 Yishun Industrial Park A, Singapore 768743",4.1,...,None,None,None,None,None,None,None,None,None,768743
20847,20847,Mr Bean (Khoo Teck Puat Hospital),"['Desserts', 'Drinks', 'Snacks']",Desserts,Drinks,Snacks,Kiosk or Stall,#01-08 Khoo Teck Puat Hospital Yishun Central ...,NaN,NaN,...,None,None,None,None,None,None,None,None,None,768828


In [17]:
final_data = data.drop(data[data["Travelling from Central Area"].isna()].index)

In [18]:
infile = open('data with distance.pkl','rb')
distData = pickle.load(infile)
infile.close()

In [19]:
distData

,Unnamed: 0,Stall Name,Cuisine,Cuisine 1,Cuisine 2,Cuisine 3,Shop Type,Address,Google Address,Average Review,...,Travelling from Hougang,Travelling from Ang Mo Kio,Travelling from Sengkang,Travelling from Clementi,Travelling from Serangoon,Travelling from Yishun,Travelling from Pasir Ris,Travelling from Queenstown,Travelling from Central Area,postal
0,0,Fatty Cheong,[None],NaN,NaN,NaN,Kiosk or Stall,#01-144 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-120, Singapore 150006",4.2,...,<__main__.travel_data object at 0x7fae7c565f70>,<__main__.travel_data object at 0x7fae7c771130>,<__main__.travel_data object at 0x7fae7c978370>,<__main__.travel_data object at 0x7fae7cb803d0>,<__main__.travel_data object at 0x7fae5d1a4490>,<__main__.travel_data object at 0x7fae5d3ab790>,<__main__.travel_data object at 0x7fae5d5b3970>,<__main__.travel_data object at 0x7fae5d7ba8b0>,<__main__.travel_data object at 0x7fae5d9c29d0>,150006
1,1,Mani Iyer Mess,['South Indian'],South Indian,NaN,NaN,Restaurant,#01-01 Little India Conservation Area Hindoo R...,Hindoo Rd,4.3,...,<__main__.travel_data object at 0x7fae7c565fd0>,<__main__.travel_data object at 0x7fae7c771190>,<__main__.travel_data object at 0x7fae7c9783d0>,<__main__.travel_data object at 0x7fae7cb80430>,<__main__.travel_data object at 0x7fae5d1a44f0>,<__main__.travel_data object at 0x7fae5d3ab7f0>,<__main__.travel_data object at 0x7fae5d5b39d0>,<__main__.travel_data object at 0x7fae5d7ba910>,<__main__.travel_data object at 0x7fae5d9c2a30>,209107
2,2,Stall #01-46,"['Chinese', 'Snacks']",Chinese,Snacks,NaN,Kiosk or Stall,#01-46 Jalan Batu Food Centre Jalan Batu 432004,NaN,NaN,...,<__main__.travel_data object at 0x7fae7c56a070>,<__main__.travel_data object at 0x7fae7c7711f0>,<__main__.travel_data object at 0x7fae7c978430>,<__main__.travel_data object at 0x7fae7cb80490>,<__main__.travel_data object at 0x7fae5d1a4550>,<__main__.travel_data object at 0x7fae5d3ab850>,<__main__.travel_data object at 0x7fae5d5b3a30>,<__main__.travel_data object at 0x7fae5d7ba970>,<__main__.travel_data object at 0x7fae5d9c2a90>,432004
3,3,Wei Xiang Kitchen,"['Chinese', 'Seafood']",Chinese,Seafood,NaN,Kiosk or Stall,#01-106 ABC Brickworks Food Centre Jalan Bukit...,"6 Jalan Bukit Merah, #01-106, Singapore 150006",5.0,...,<__main__.travel_data object at 0x7fae7c565f70>,<__main__.travel_data object at 0x7fae7c771130>,<__main__.travel_data object at 0x7fae7c978370>,<__main__.travel_data object at 0x7fae7cb803d0>,<__main__.travel_data object at 0x7fae5d1a4490>,<__main__.travel_data object at 0x7fae5d3ab790>,<__main__.travel_data object at 0x7fae5d5b3970>,<__main__.travel_data object at 0x7fae5d7ba8b0>,<__main__.travel_data object at 0x7fae5d9c29d0>,150006
4,4,The Thai Table,"['Asian', 'Indochinese', 'Thai']",Asian,Indochinese,Thai,Restaurant,Picardy Garden Jalan Pari Burong 488686,"20 Jln Pari Burong, Singapore 488686",4.2,...,<__main__.travel_data object at 0x7fae7c56a0d0>,<__main__.travel_data object at 0x7fae7c771250>,<__main__.travel_data object at 0x7fae7c978490>,<__main__.travel_data object at 0x7fae7cb804f0>,<__main__.travel_data object at 0x7fae5d1a45b0>,<__main__.travel_data object at 0x7fae5d3ab8b0>,<__main__.travel_data object at 0x7fae5d5b3a90>,<__main__.travel_data object at 0x7fae5d7ba9d0>,<__main__.travel_data object at 0x7fae5d9c2af0>,488686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20844,20844,Snackz It! (Vivo City),"['Asian', 'Local', 'Snacks']",Asian,Local,Snacks,Confectionery or Bakery,#B2-K24 Vivocity Harbourfront Walk 098585,"1 HarbourFront Walk, #B2-K24 VivoCity, Singapo...",2.0,...,<__main__.travel_data object at 0x7fae7c582df0>,<__main__.travel_data object at 0x7fae7c78af10>,<__main__.travel_data object at 0x7fae7c9961f0>,<__main__.travel_data object at 0x7fae7cb9e1f0>,<__main__.travel_data object at 0x7fae5d1c1310>,<__main__.travel_data object at 0x7fae5d3c8610>,<__main__.travel_data object at 0x7fae5d5d1790>,<__main__.travel_data obj